In [37]:
import os
hr_path = 'E:\\dl_data\\SR\\open_sr\\train\\hr\\'

new_hr_path = 'E:\\dl_data\\SR\\open_sr_gen\\hr\\'
new_lr_path = 'E:\\dl_data\\SR\\open_sr_gen\\lr\\'
new_hr_valid_path = 'E:\\dl_data\\SR\\open_sr_gen\\hr_valid\\'
new_lr_valid_path = 'E:\\dl_data\\SR\\open_sr_gen\\lr_valid\\'
path_splitter = '\\'

os.makedirs(new_hr_path, exist_ok=True)
os.makedirs(new_lr_path, exist_ok=True)
os.makedirs(new_hr_valid_path, exist_ok=True)
os.makedirs(new_lr_valid_path, exist_ok=True)

target_width = 512
target_height = 512

In [44]:
target_width = 512
target_height = 512

from PIL import Image
from glob import glob
import os, shutil
from tqdm import tqdm

import numpy as np
import cv2

# remove old files
filelist = glob(os.path.join(new_hr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_lr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_hr_valid_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_lr_valid_path, "*"))
for f in filelist:
    os.remove(f)

# variables
total_count = 0
designated_valid_count = 12
current_count = 0
do_each = False

#
filelist = glob(os.path.join(hr_path, "*.png"))
if total_count == 0:
    total_count = len(filelist)
    
for _, file in enumerate(tqdm(filelist)):
    filename = file.split(path_splitter)[-1]
    filename_without_ext = filename.split('.')[0]   

    ### resize or filter
    modes = ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos', 'blur', 'gaussian_blur', 'median_blur', 'bilateral_filter']
    applys = [True, False, True, False, True, True, False, True, False, True]
    
    #
    do_modes = []
    for index, mode in enumerate(modes):
        if applys[index] is True:
            do_modes.append(mode)
    if do_each is False: # 파일마다 한번씩만 round robin
        do_mode = do_modes[current_count % len(do_modes)]
        do_modes.clear()
        do_modes.append(do_mode)
        
    # train or valid
    for_valid = False
    if current_count + designated_valid_count >= total_count:
        for_valid = True

    target_hr_path = new_hr_path
    target_lr_path = new_lr_path
    if for_valid is True:
        target_hr_path = new_hr_valid_path
        target_lr_path = new_lr_valid_path
    
    for mode in do_modes:
        if mode in ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos']:
            # resize
            img = Image.open(file)
            if mode == 'nearest':
                img_resize = img.resize((target_width, target_height), Image.Resampling.NEAREST)
            elif mode == 'box':
                img_resize = img.resize((target_width, target_height), Image.Resampling.BOX)
            elif mode == 'bilinear':
                img_resize = img.resize((target_width, target_height), Image.Resampling.BILINEAR)
            elif mode == 'hamming':
                img_resize = img.resize((target_width, target_height), Image.Resampling.HAMMING)
            elif mode == 'bicubic':
                img_resize = img.resize((target_width, target_height), Image.Resampling.BICUBIC)
            elif mode == 'lanczos':
                img_resize = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
                
            # copy hr
            new_file_hr = target_hr_path + filename_without_ext + "_" + mode + ".png"
            shutil.copy(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + filename_without_ext + "_" + mode + ".png"
            img_resize.save(new_file_lr)
        
            #print(filename + ' ' + mode + ' done')
        else:
            img = cv2.imread(file)
            resize_img = cv2.resize(img, (target_width, target_height))   
            if mode == 'blur':
                dst = cv2.blur(resize_img, (5,5))
            elif mode == 'gaussian_blur':
                dst = cv2.GaussianBlur(resize_img, (5,5), 0)
            elif mode == 'median_blur':
                dst = cv2.medianBlur(resize_img, 5)
            elif mode == 'bilateral_filter':
                dst = cv2.bilateralFilter(resize_img, 9, 75, 75)
            else:
                continue
            
            # copy hr
            new_file_hr = target_hr_path + filename_without_ext + "_" + mode + ".png"
            shutil.copy(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + filename_without_ext + "_" + mode + ".png"
            cv2.imwrite(new_file_lr, dst)

            #print(filename + ' ' + mode + ' done')
    
    current_count += 1
    #if current_count > 10:
    #    break
    
print('all done')

### 컨볼루션
#img = cv2.imread(hr_path + filename)
#resize_img = cv2.resize(img, (target_width, target_height))
#kernel = np.ones((5,5),np.float32)/25
#dst = cv2.filter2D(resize_img, -1, kernel)
#new_file = lr_path + '7_convolution_' + filename
#cv2.imwrite(new_file, dst)

100%|██████████████████████████████████████████████████████████████████████████████| 1640/1640 [04:39<00:00,  5.86it/s]

all done
